# 1. Алгоритмы выравнивания

## 1.1 Алгоритм Нидлмана-Вунша

In [19]:
import numpy as np
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1):
    m, n = len(seq1), len(seq2)
    #1 Матриц
    score_matrix = np.zeros((m + 1, n + 1))
    
    # 2 Заполняем первые строки и столбцы
    for i in range(m + 1):
        score_matrix[i][0] = i * gap_penalty
    for j in range(n + 1):
        score_matrix[0][j] = j * gap_penalty
    
    #3 Алгоритм
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)
    
    # Трассировка (backtracking)
    aligned_seq1, aligned_seq2 = "", ""
    i, j = m, n
    
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        if i > 0 and j > 0 and (seq1[i - 1] == seq2[j - 1] and current_score == score_matrix[i - 1][j - 1] + match_score or 
                                 seq1[i - 1] != seq2[j - 1] and current_score == score_matrix[i - 1][j - 1] + mismatch_score):
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif i > 0 and current_score == score_matrix[i - 1][j] + gap_penalty:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
    
    return aligned_seq1, aligned_seq2, score_matrix

seq1 = "CCGCAAGTTCGATATCAAATCGCTTGGATTGTCTAAGATACGCGCCCCGCATCCCGTATTAAGGTGGGAGGGGGTTTATGACTGTTCTCGCGCACAACTC"
seq2 = "CTCAATCAAAAGGCTAAAAGCAAGTTCAGGTCTACGCATCTACGATTGATCTGCTCCCTTCTGTCGATTAATTGTATCCACTAGAGCCAGCGCCCTCATC"
aligned_seq1, aligned_seq2, score_matrix = needleman_wunsch(seq1, seq2)
print("Выравнивание:")
print(aligned_seq1)
print(aligned_seq2)

Выравнивание:
CCGCAAGTTCGATA---TCAAATC--GCTTGGATTGTCTAAG-A--TACG--CGCCCCGCATCCCGTAT-TAAGGTGGGAGGGGGTT-TAT-GACT-GTTCTC-GCGCAC-AACTC
-CTCAA--TCAAAAGGCTAAAAGCAAG-TT-CA-GGTCTACGCATCTACGATTGATCTGC-TCCC-T-TCT---GT-CGA-TTAATTGTATCCACTAGAGC-CAGCGCCCTCA-TC


## Пример


In [20]:
seq1 = "TCGGTCCCCTATCAAGCGGAGCATTTGGGCTGTTTATTGGTCGGAGAGGCGCGCTTTAACTTGGGTAAATGGTTGGGGCACTTTCCCGGCTCGGGGGATATCTGAAGGTGAGTAGGCAACGCGGTTATTTAGGCAAGCCAGTTGTCGTGCATGAGGTTGACGGGGTAGAGCGAGACAGGTGAAGTCTCGGAATGGATTCATG"
seq2 = "CGCTTCTACGTCGCCACGTCGAGCCTCAGCATAACCAGCCACGTTTGCGTTACAATGAAACATCTTCTGGTCCGTATTCTGTAAAGCCGCACGCTTAATGAGCAATGTTGCCGGCCTTCTCCCGGGAAGTGGTGACGGTCCCGGCTGGAGCCGAGTGACCGCGAGTGGTTCTGTGTTCCACACGGTGTTTGGCGGTAGTGC"
aligned_seq1, aligned_seq2, score_matrix = needleman_wunsch(seq1, seq2)
print("Выравнивание:")
print(aligned_seq1)
print(aligned_seq2)

Выравнивание:
----TC--GGTC-CC-C-T--A---TCAAGC----GGAG-CA--TTTG-G--GC--TG-----T-TTATTGGT-CG------G-AGAG--GCGCGCTTTAACTTGGGTAAATGGTTG-GGGCACTTTCCCGGCTCGGGGGATATCTGAAGGTGA-GTAGGCAACGCGGTTATTTAGGCAAGCC-AGTTG-TCGTGCATGAGGTTGACGGGGTAGAGCGAGACAGGTGAAGTCTCGGAATGGATTCA-TG-
CGCTTCTACGTCGCCACGTCGAGCCTC-AGCATAACCAGCCACGTTTGCGTTACAATGAAACATCTT-CTGGTCCGTATTCTGTAAAGCCGCACGC-TTAA--TGAG-CAAT-GTTGCCGGC-C-TTCTC--C-C--GGGA-A---G-TGGTGACG--GTC--C-CGG----CT-GG--AGCCGAG-TGACCGCG-A-GTGGTT--C--TGT-GTTCCACAC-GGT---GT-TTGG--CGG--T-AGTGC


## 1.2 Алгоритм Смитта-Ватермана и пример


In [23]:
def smith_waterman(seq1, seq2, match_score=3, mismatch_score=-3, gap_penalty=-2):
    m, n = len(seq1), len(seq2)
    score_matrix = np.zeros((m + 1, n + 1))  
    
    max_score = 0  # Максимальный балл в матрице
    max_pos = (0, 0)  # Координаты максимального балла
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            
            # Отличие от Нидлмана-Вунша:
            score_matrix[i][j] = max(match, delete, insert, 0)
            
            # 🐤Запоминаем наибольшее значение и его координаты
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)

    aligned_seq1, aligned_seq2 = "", ""
    i, j = max_pos

    while i > 0 and j > 0 and score_matrix[i][j] > 0:
        if score_matrix[i][j] == score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score):
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif score_matrix[i][j] == score_matrix[i - 1][j] + gap_penalty:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2, max_score
seq1 = "CCGCAAGTTCGATATCAAATCGCTTGGATTGTCTAAGATACGCGCCCCGCATCCCGTATTAAGGTGGGAGGGGGTTTATGACTGTTCTCGCGCACAACTC"
seq2 = "CTCAATCAAAAGGCTAAAAGCAAGTTCAGGTCTACGCATCTACGATTGATCTGCTCCCTTCTGTCGATTAATTGTATCCACTAGAGCCAGCGCCCTCATC"
aligned_seq1, aligned_seq2, max_score = smith_waterman(seq1, seq2)

print("😝 Наилучшее локальное выравнивание:")
print(aligned_seq1)
print(aligned_seq2)
print("👀 Максимальный балл:", max_score)

😝 Наилучшее локальное выравнивание:
GCAAGTTCGATATCAAATCGC-T-T-GGATTG-TCTAAGATACGCGC-CCCGCATCCCGTATTAAGGTGGGAGGGGGTTTAT-GACT-GTTCTC-GCGCACAACTC
GCAAGTTC-AGGTC-TA-CGCATCTACGATTGATCT--GCT-C-C-CTTCTG--T--CG-ATTAA--T---------TGTATCCACTAGAGC-CAGCGC-C--CTC
👀 Максимальный балл: 72.0
